## Import Libraries

In [ ]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.utils.data as Data
from torch import optim

In [ ]:
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as preprocessing
import numpy as np
import pandas as pd
from scipy import signal
import time
from utils import *

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')

## Define Network Structure

In [ ]:
class TCN(nn.Module):
    def __init__(self):
        super(TCN, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=4, kernel_size=2, stride=1, dilation=1)
        self.batch_norm1 = nn.BatchNorm1d(4)
        self.conv2 = nn.Conv1d(in_channels=4, out_channels=8, kernel_size=2, stride=1, dilation=2)
        self.batch_norm2 = nn.BatchNorm1d(8)
        self.conv3 = nn.Conv1d(in_channels=8, out_channels=16, kernel_size=2, stride=1, dilation=4)
        self.linear = nn.Linear(16 * 73, 12)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.batch_norm1(x)
        x = F.relu(self.conv2(x))
        x = self.batch_norm2(x)
        x = F.relu(self.conv3(x))
        x = x.view(-1, 16 * 73)
        x = self.linear(x)
        
        return x
    
    def parameter_count(self):
        par = list(self.parameters())
        s = sum([np.prod(list(d.size())) for d in par])
        return s

## Preprocess

### load dataset

In [ ]:
dataset = np.load('../dataset/pems.npy', allow_pickle=True).item()
x, y = dataset['X'], dataset['Y']

### Z-Score Normalization

In [ ]:
x = x.reshape(x.shape[0], x.shape[2])
scaler = preprocessing.StandardScaler().fit(x)
x = scaler.transform(x)
x = x.reshape(x.shape[0], 1, x.shape[1])

### Split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
y_train = y_train.reshape(y_train.shape[0], y_train.shape[2])
y_test = y_test.reshape(y_test.shape[0], y_test.shape[2])

### Batch

In [ ]:
dataset_train = Data.TensorDataset(x_train, y_train)
loader = Data.DataLoader(dataset=dataset_train, batch_size=256, shuffle=True, num_workers=2)

batch_num = 0
for step, (batch_x, batch_y) in enumerate(loader):
    batch_num += 1
print('%d batches' % batch_num)

## Run Model

### Choose Device

In [ ]:
device = choose_device()
print(device)

### Multiple Runs

In [ ]:
def evaluate(x, y, net, criterion=nn.MSELoss(), clips=12, suppress_output=False):
    y_hat = net(x.to(device))
    y_hat = y_hat[:, :clips]
    y = y[:, :clips].to(device)
    loss = criterion(y_hat, y)
    RMSE = loss.item() ** 0.5
    MAPE = compute_MAPE(y, y_hat)
    MAE = compute_MAE(y, y_hat)
    if suppress_output == False:
        print('samples: %d - %d\ntime clips: %d\nRMSE: %.2f\nMAPE: %.2f%%\nMAE: %.2f' % (y_hat.shape[0], y.shape[0], clips, RMSE, MAPE, MAE))
    return RMSE, MAPE, MAE

In [ ]:
def run(rounds=20, epochs=150, lr=0.001, weight_decay=0.001, clips=[3, 6, 12], criterion=nn.MSELoss()):
    # init
    criterions = ['RMSE', 'MAPE', 'MAE']
    columns = []
    for clip in clips:
        for each in criterions:
            columns.append('%s_%d' % (each, clip))
    df = []
    # multiple runs
    for cur_round in range(1, rounds + 1, 1):
        # init
        net = TCN().to(device)        
        optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)
        train_losses, test_losses = [], []
        # train
        for epoch in range(epochs):
            t_start = time.time()
            running_RMSE, running_MAPE = 0, 0

            for step, (batch_x, batch_y) in enumerate(loader):
                input_x = batch_x.to(device)
                y = batch_y.to(device)  

                optimizer.zero_grad()

                output_data = net(input_x)
                loss = criterion(output_data, y)
                MAPE = compute_MAPE(output_data, y)

                loss.backward()
                optimizer.step()

                running_RMSE += loss.item() ** 0.5
                running_MAPE += MAPE
            train_losses.append(running_RMSE / batch_num)

            test_losses.append(criterion(net(x_test.to(device)), y_test.to(device)).item() ** 0.5)
            t_end = time.time()

            print('\rround=%02d, epoch=%d, RMSE=%.2f, MAPE=%.2f%%, time=%.2fs per epoch      ' \
                  % (cur_round, epoch+1, running_RMSE / batch_num, running_MAPE / batch_num, t_end-t_start), end='')
        print()
        #evaluate
        evaluation_clip = []
        for clip in clips:
            RMSE, MAPE, MAE = evaluate(x_test, y_test, net, criterion, clips=clip, suppress_output=True)
            eva = [RMSE, MAPE, MAE]
            evaluation_clip += eva
        df.append(evaluation_clip)
    df = pd.DataFrame(df, columns=columns)
    df.index = np.arange(1, rounds+1, 1)
    return df, train_losses, test_losses

In [ ]:
df, train_losses, test_losses = run(rounds=10, epochs=150, lr=0.006, weight_decay=0.0001)

### Plot Loss Iteration

In [ ]:
dloss = pd.DataFrame({'epoch': np.arange(1, len(train_losses)+1, 1), 'train_loss': train_losses, 'test_loss': test_losses})
sns.lineplot(x='epoch', y='train_loss', data=dloss, label='train loss')
sns.lineplot(x='epoch', y='test_loss', data=dloss, label='test loss')
plt.ylabel('value')
plt.legend()
plt.show()

## Evaluation

### Average Result

In [ ]:
df.describe()

### Plot Variance

In [ ]:
sns.boxplot(data=df)
plt.show()

### Save Result

In [ ]:
df.to_csv('../Multiple Run Results/TCN_10.csv', index=False)